In [2]:
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.workspace = "C:/Users/Nina/Desktop/Final_project"


#Reprojecting building footprints into parcel file projection
#creating spatial reference from parcel file which is already projected
prjfile = "Parcels_AOI.prj"
spatialref = arcpy.SpatialReference(prjfile)
myref = spatialref.name

#Process: Project
"""input_features = "footprints_AOI.shp"
output_feature_class = "Footprints_proj"
out_coordinate_system = arcpy.SpatialReference(prjfile)"""
arcpy.Project_management("footprints_AOI.shp", "Footprints_proj", arcpy.SpatialReference(prjfile))


#Process: Intersect
inFeatures = ["Parcels_AOI.shp", "Footprints_proj.shp"]
intersectOutput = "intersect.shp"
arcpy.Intersect_analysis(inFeatures, intersectOutput, "ALL", "", "INPUT")

# Process: Add Field
arcpy.AddField_management("intersect.shp", "new_area", "LONG", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")

# Process: Calculate Field - calculate area of intersected bf and parcels
arcpy.CalculateField_management("intersect.shp", "new_area", "!shape.area!", "PYTHON", "")

# Process: Summary Statistics - find max area for new area field
arcpy.Statistics_analysis ("intersect.shp", "summary", [["new_area", "MAX"]], "BF_ID")

#Process join by BF ID summary table to intersected BF and parcels (to get PID)
max_area_joined = arcpy.AddJoin_management ("intersect.shp", "BF_ID", "summary", "BF_ID", "")

#Create new shpfile from join above
arcpy.CopyFeatures_management(max_area_joined, "AreaJoin")



#add field to new shp file
arcpy.AddField_management("AreaJoin.shp", "subtract", "LONG", "", "", "", "", "NULLABLE", "NON_REQUIRED", "")

#Process: Calculate Field - subtract original area from new area in new field
arcpy.CalculateField_management("AreaJoin.shp", "subtract", "!interse_55! - !summary_MA!", "PYTHON", "")

# Execute Select
arcpy.Select_analysis("AreaJoin.shp", "Final_fp.shp", '"subtract" = 0')





ExecuteError: ERROR 999999: Something unexpected caused the tool to fail. Please refer to the error help for potential solutions, or contact Esri Technical Support http://esriurl.com/support.
Create output feature class failed
Failed to execute (Project).
